In [ ]:
import os
import json
import boto3

### Update Lambda Function

Arthur-Only, will make public

#### TODO: REMOVE!!!!

In [ ]:
! zip lambda-notebook-trigger-deployment.zip lambda_function.py

In [ ]:
! aws s3 cp ./lambda-notebook-trigger-deployment.zip s3://s3-bucket-arthur-public/mars/lambda-notebook-trigger-deployment.zip

## Create Notebook Docker Images

In [ ]:
# if needed, set your ECR_REGISTRY_URL env var

# os.environ['ECR_REGISTRY_URL'] = "<YOUR-REGISTRY>"
# os.environ['AWS_REGION'] = "<YOUR-REGION>"

In [ ]:
# create an ECR repository for this image

! aws ecr create-repository --repository-name sagemaker-notebook --image-scanning-configuration scanOnPush=true --image-tag-mutability MUTABLE

In [ ]:
#  build the image locally

! docker build ../ -f ./sagemaker.Dockerfile -t sagemaker-notebook

In [ ]:
# re-tag the image with the AWS registry

! docker tag sagemaker-notebook:latest ${ECR_REGISTRY_URL}/sagemaker-notebook:latest

In [ ]:
# login to ecr registry

! aws ecr get-login-password --region ${AWS_REGION} | docker login --username AWS --password-stdin ${ECR_REGISTRY_URL}

In [ ]:
# push the image up

! docker push ${ECR_REGISTRY_URL}/sagemaker-notebook:latest

## Create AWS Infrastructure

In [ ]:
# ensure you have these environment variables set:

# os.environ['ARTHUR_ENDPOINT_URL'] = "https://app.arthur.ai"
# os.environ['ARTHUR_API_KEY'] = "<YOUR-ARTHUR-API-KEY>"
# os.environ['NASA_API_KEY'] = "<YOUR-NASA-API-KEY>"

In [ ]:
! aws cloudformation create-stack --stack-name sagemaker-run-arthur-mars-notebook \
    --template-body file://$(pwd)/cloudformation.yml --capabilities CAPABILITY_NAMED_IAM \
    --parameters ParameterKey=ArthurURL,ParameterValue=${ARTHUR_ENDPOINT_URL} \
    ParameterKey=ArthurApiKey,ParameterValue=${ARTHUR_API_KEY} \
    ParameterKey=NasaApiKey,ParameterValue=${NASA_API_KEY}

## Upload Project to S3

In [ ]:
# update me!
S3_BUCKET = "arthur-sagemaker"

os.environ['S3_BUCKET'] = S3_BUCKET

In [ ]:
# create directories for our api data, reference data, model, and output notebooks
! aws s3 cp ./.empty s3://${SAGEMAKER_S3_BUCKET}/mars/api-data/.empty
! aws s3 cp ./.empty s3://${SAGEMAKER_S3_BUCKET}/mars/reference-data/.empty
! aws s3 cp ./.empty s3://${SAGEMAKER_S3_BUCKET}/mars/model/.empty
! aws s3 cp ./.empty s3://${SAGEMAKER_S3_BUCKET}/mars/output_notebooks/.empty

# copy predict function
! aws s3 cp ../predict.py s3://${SAGEMAKER_S3_BUCKET}/mars/predict.py
    
# copy util folder
! aws s3 cp ../util s3://${SAGEMAKER_S3_BUCKET}/mars/util --recursive --exclude "*" --include "*.py"

# copy notebook
! aws s3 cp ../Quickstart.ipynb s3://${SAGEMAKER_S3_BUCKET}/mars/MarsNotebook.ipynb
    
# copy api key
! echo ${ARTHUR_API_KEY} > ./arthur-api-key.txt
! aws s3 cp ./arthur-api-key.txt s3://${SAGEMAKER_S3_BUCKET}/mars/arthur-api-key.txt

In [ ]:
# optional: if you've downloaded the model you can upload it to S3 so that each job doesn't have to

! aws s3 cp ../model/model_weights.pt s3://${SAGEMAKER_S3_BUCKET}/mars/model/model_weights.pt

## Invoking Runs

In [ ]:
# update me!
MARS_MODEL_ID = "fill in"

In [ ]:
base_lambda_payload = {
    "s3_path": f"s3://{S3_BUCKET}/mars",
    "image": f"{os.environ['ECR_REGISTRY_URL']}/sagemaker-notebook:latest",
    "notebook": "MarsNotebook.ipynb",
    "instance_type": "ml.c5.2xlarge",
}

### Trigger Manual Run

In [ ]:
import boto3

lambda_client = boto3.client('lambda')

In [ ]:
manual_lambda_payload = {
    "parameters": {
        "lookback_days": 30,

        #  if you want to add historical data to an already-created model, specify your model ID above
        #  if you want this manual run to create a new model, you can comment or remove this line
        "mars_model_id": MARS_MODEL_ID
    },
    **base_lambda_payload
}

In [ ]:
lambda_client.invoke(FunctionName='RunArthurMarsNotebook',
              Payload=bytes(json.dumps(manual_lambda_payload), encoding='utf-8'))

### Schedule Daily Runs

If you haven't filled in "MARS_MODEL_ID" yet and used the manual run to create a new model, find your model ID in the Arthur dashboard in fill in that value now

In [ ]:
daily_lambda_payload = {
    "parameters": {
        "lookback_days": 5,
        "mars_model_id": MARS_MODEL_ID
    },
    **base_lambda_payload
}

In [ ]:
# fetch our AWS Account ID to use with EventBridge

AWS_ACCOUNT_ID = boto3.client("sts").get_caller_identity()['Account']
os.environ['AWS_ACCOUNT_ID'] = AWS_ACCOUNT_ID

In [ ]:
events_client = boto3.client("events")

In [ ]:
# create an event rule

events_client.put_rule(Name="RunArthurMarsNotebook-daily", ScheduleExpression="cron(30 10 * * ? *)",
                       Description="Daily run of the Arthur Mars notebook")

In [ ]:
# add a permission for this event rule to trigger the lambda

! AWS_ACCOUNT_ID=`aws sts get-caller-identity --query Account --output text` && \
  aws lambda add-permission --statement-id DailyRun-RunArthurMarsNotebook \
              --action lambda:InvokeFunction \
              --function-name RunArthurMarsNotebook \
              --principal events.amazonaws.com \
              --source-arn arn:aws:events:${AWS_REGION}:${AWS_ACCOUNT_ID}:rule/RunArthurMarsNotebook-daily


In [ ]:
# assign our lambda as the target for the event

targets = [{
    "Id": "Default",
    "Arn": f"arn:aws:lambda:{os.environ['AWS_REGION']}:{AWS_ACCOUNT_ID}:function:RunArthurMarsNotebook",
    "Input": json.dumps(daily_lambda_payload)
}]

events_client.put_targets(Rule="RunArthurMarsNotebook-daily", Targets=targets)